In [3]:
import urllib2
import os
import zipfile
import pandas as pd
import json
import ast
import numpy as np

In [ ]:
# download data
def retrieveData():
    url = 'http://crowdcrafting.org/project/landfill/tasks/export?type=task_run&format=csv'
    response = urllib2.urlopen(url)
    output = open('landfill_data.zip', 'wb')
    output.write(response.read())
    output.close()    
    return unzip()

def unzip():
    fh = open('landfill_data.zip', 'rb')
    z = zipfile.ZipFile(fh)
    for name in z.namelist():
        outpath = ""
        z.extract(name, outpath)
    fh.close()
    os.remove('landfill_data.zip')
    
    return load_data(name)

In [91]:
data = pd.read_csv('../landfill_task_run.csv', error_bad_lines=False)

Skipping line 642: expected 20 fields, saw 21



In [92]:
data.shape

(1571, 20)

In [93]:
# do not export things without geometries
d = data[data['task_runinfo__area'] != '0']

In [94]:
d.shape

(661, 20)

In [95]:
def create_features(data):
    record = ast.literal_eval(data[0])
    for i in record:
        geometry = ast.literal_eval(i['area_geometry'])
        for point in geometry['coordinates']:
            result = {
                "type": "Feature",
                "geometry": geometry,
                "properties": {
                    "area" : i['area'], 
                    "id" : i['id'], 
                    "fac_name" : data[1],
                    "fac_state" : data[2],
                    "fac_street" : data[3],
                    "task_id" : data[4],
                    "task_user_id" : data[5],
                    "task_runinfo__skip" : data[6],
                }
            }
            return result                

In [103]:
#a stupid hack - zip all of these lists and pass them
# into the create_feature function as variable temp
temp = zip(
            list(d['task_runinfo__area']),
            list(d['task_runinfo__Facility_Name']),
            list(d['task_runinfo__Facility_State']),
            list(d['task_runinfo__Facility_Street']),
            list(d['task_run__task_id']),
            list(d['task_run__user_id']),
            list(d['task_runinfo__skip']),    
          )

features = list(map(create_features, temp))

for index,feature in enumerate(features):
    if feature['properties']['area'] == 0:
        del features[index]
        
geojson = json.dumps({
    "type": "FeatureCollection",
    "features": features,
})

In [104]:
len(features)

658

In [99]:
for i,j in enumerate(features):
    if j['properties']['area'] == 0:
        del features[i]

In [100]:
len(features)

658

In [101]:
def writeFile(geojson):
    print "Writing geojson file..."
    outputFile = open('../landfill_geojson.geojson', 'wb')
    outputFile.write(geojson)
    outputFile.close()

In [105]:
writeFile(geojson)

Writing geojson file...
